In [1]:
import pandas as pd
import string

URL = 'http://www.treasury.gov/ofac/downloads/sdn.csv'

NULL = '-0-'

COLS = [
    "ent_num",
    "SDN_Name",
    "SDN_Type",
    "Program",
    "Title",
    "Call_Sign",
    "Vess_type",
    "Tonnage",
    "GRT",
    "Vess_flag",
    "Vess_owner",
    "remarks",
]

def retrieve_sdn_list():
    df = pd.read_csv(URL, names=COLS)
    df = filter_individuals(df)
    df['full_name'] = df['SDN_Name'].map(format_name)
    return df

def filter_individuals(df):
    return df.ix[df['SDN_Type'] == 'individual']

def format_name(last_name_first_name):
    try:
        last_name, first_name = last_name_first_name.split(', ', 1)
        return "%s %s"%(first_name.lower(), last_name.lower())
    except:
        return last_name_first_name.lower()

def retrieve_and_save_to_csv(filename='individual_terrorists.csv'):
    df = retrieve_sdn_list()
    return df.to_csv(filename, index=False)


def retrieve_shop_account_owners_rdd(sc):
    return sc.jsonFile('/data/frontroom/dimensions/shop_dimension/{{latest}}')\
             .map(lambda r: r.keep("Shop Account Owner Name", "Shop ID", "Shop Country"))\
             .map(lambda r: r.set("Shop Account Owner Name", r["Shop Account Owner Name"].lower()))


def run(sc):
    df = retrieve_sdn_list()
    terrorist_list = list(df['full_name'])
    account_owners_rdd = retrieve_shop_account_owners_rdd(sc)
    results = account_owners_rdd.filter(lambda r: r["Shop Account Owner Name"] in terrorist_list)
    return map( lambda r: r[1][1].as_dict(), results)






In [ ]:
results = run(sc)

In [3]:
results

1840.013: [GC [PSYoungGen: 142172K->10760K(140288K)] 142276K->10872K(489984K), 0.0180890 secs] [Times: user=0.30 sys=0.00, real=0.01 secs] 
2436.492: [GC [PSYoungGen: 139784K->8592K(149504K)] 139896K->8712K(499200K), 0.0124640 secs] [Times: user=0.11 sys=0.00, real=0.01 secs] 


NameError: name 'results' is not defined